In [1]:
using Pkg
# Uncomment the next line if installing Julia for the first time
# Pkg.Registry.add("General")
Pkg.activate("..")
Pkg.status()

  Activating project at `~/Documents/GitHub-2/ACESIDopt.jl`


Project ACESIDopt v0.1.0
Status `~/Documents/GitHub-2/ACESIDopt.jl/Project.toml`
⌃ [3b96b61c] ACEpotentials v0.10.0
  [3da9722f] ASEconvert v0.2.0
  [a963bdd2] AtomsBase v0.5.2
⌃ [f5cc8831] AtomsBuilder v0.2.2
  [a3e0e189] AtomsCalculators v0.2.3
  [9855a07e] AtomsCalculatorsUtilities v0.1.8
  [352459e4] ExtXYZ v0.2.2
  [7073ff75] IJulia v1.33.0
  [b964fa9f] LaTeXStrings v1.4.0
  [98b081ad] Literate v2.21.0
⌅ [aa0f7f06] Molly v0.22.3
  [6f286f6a] MultivariateStats v0.10.3
⌃ [91a5bcdd] Plots v1.41.3
⌅ [08abe8d2] PrettyTables v2.4.0
⌃ [6099a3de] PythonCall v0.9.30
⌃ [90137ffa] StaticArrays v1.9.15
  [fd094767] Suppressor v0.2.8
  [1986cc42] Unitful v1.27.0
  [8ba89e20] Distributed v1.11.0
  [37e2e46d] LinearAlgebra v1.11.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [14]:
using LaTeXStrings, MultivariateStats, Plots, PrettyTables, Printf,
      Statistics, Suppressor, ExtXYZ, Unitful

using ACEpotentials, AtomsBase

In [15]:

mace_dataset = ExtXYZ.load("/Users/msachs2/Documents/GitHub-2/testASECalculators/results/glycine_remd_parallel2/replica_0_extxyz_100frames.xyz")
system = mace_dataset[2]
AtomsBase.position(system, 1)
fieldnames(typeof(system.atom_data))
system.atom_data.forces
system.system_data

(energy = -58.15094757080078, periodicity = (true, true, true), cell_vectors = Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}[[33.626 Å, 0.0 Å, 0.0 Å], [0.0 Å, 34.122 Å, 0.0 Å], [0.0 Å, 0.0 Å, 31.78 Å]])

In [32]:
mace_dataset[1] 

Atoms(C₂H₅NO₂, periodicity = TTT):
    cell_vectors      : [  33.626        0        0;
                                0   34.122        0;
                                0        0    31.78]u"Å"
    energy            : -57.841278076171875

    AtomView(N,  [ 15.3841,  15.9171,  15.8906]u"Å")
    AtomView(C,  [ 16.8305,   15.912,  15.8893]u"Å")
    AtomView(C,  [ 17.4151,  17.1866,  15.8891]u"Å")
    AtomView(O,  [ 18.6247,  17.3734,  15.8899]u"Å")
    AtomView(O,  [ 16.5734,  18.2167,  15.8914]u"Å")
    AtomView(H,  [ 15.0111,  16.8395,  15.8933]u"Å")
    AtomView(H,  [  14.986,  15.0174,  15.8913]u"Å")
    AtomView(H,  [ 17.1849,  15.3784,  16.7637]u"Å")
    AtomView(H,  [ 17.1803,  15.3538,  14.9953]u"Å")
    AtomView(H,  [ 16.9273,  19.1304,  15.8954]u"Å")


In [ ]:

r_cut = 6.0u"Å"
rnn = 2.35

rdf_tiny = ACEpotentials.get_rdf(mace_dataset, r_cut; rescale = true)
plt_rdf_1 = histogram(rdf_tiny[(:C, :C)], bins=150, label = "rdf",
                      title="mace_dataset", titlefontsize=10,
                      xlabel = L"r[\AA]", ylabel = "RDF", yticks = [],
                      xlims=(1.5,6), size=(400,200), left_margin = 2Plots.mm)
vline!(rnn * [1.0, 1.633, 1.915, 2.3, 2.5], label = "r1, r2, ...", lw=3)

rdf = ACEpotentials.get_rdf(mace_dataset, r_cut; rescale = true);
plt_rdf_2 = histogram(rdf[(:C, :C)], bins=150, label = "rdf",
                      title="mace_dataset", titlefontsize=10,
                      xlabel = L"r[\AA]", ylabel = "RDF", yticks = [],
                      xlims=(1.5,6), size=(400,200), left_margin = 2Plots.mm)
vline!(rnn * [1.0, 1.633, 1.915, 2.3, 2.5], label = "r1, r2, ...", lw=3)

plot(plt_rdf_1, plt_rdf_2, layout=(2,1), size=(400,400))

BoundsError: BoundsError: attempt to access 0-element Vector{Float64} at index [1]

In [21]:
model = ace1_model(elements = [:C, :H, :O, :N],
                   rcut = 5.5,
                   order = 3,        # body-order - 1
                   totaldegree = 8 );

In [22]:
descriptors = []
for system in mace_dataset
    struct_descriptor = sum(site_descriptors(system, model)) / length(system)
    push!(descriptors, struct_descriptor)
end
println("Computed descriptors for $(length(descriptors)) systems")

Computed descriptors for 100 systems


100-element Vector{Any}:
 [0.04479175009056713, 0.19877909920536588, 0.08189461744743798, 0.04681759619808917, -0.058298707533474305, 0.11913006361476546, -0.0099320783028919, -0.004357758754068153, 0.025063091862674126, 0.08862219365433446  …  0.0065693638702215475, 0.0, 0.05016230184431523, -0.3050241634969784, -0.01196097022261429, 0.0, -0.06542124225082886, 0.09404288413893148, -0.0033072961518870922, 0.0]
 [0.05136915922581735, 0.19533195569930714, 0.08056962871123853, 0.04825925576491981, -0.052239278544942744, 0.12056526491850542, -0.006297101988243936, -0.006237078807025523, 0.0014149078545150485, 0.09574057578125907  …  0.001036511845129567, 0.0, 0.04930867208788701, -0.2493647828582338, -0.015039128136176135, 0.0, -0.05777904155164763, 0.024261667626149324, 0.004377324448631923, 0.0]
 [0.05511299192475475, 0.18616724812683585, 0.0785541727832535, 0.046023531562293364, -0.04536005002485253, 0.12584138599658962, 0.0005926495711416734, -0.0037324748850731337, -0.0143346759061574

In [31]:
# Check the cell vectors and periodicity
system = mace_dataset[1]

println("Periodicity: ", AtomsBase.periodicity(system))
println("Cell vectors: ", get_cell_vectors(system))
println("Cell: ", AtomsBase.cell(system))

# Check the cell matrix
cell_matrix = hcat(AtomsBase.cell(system)...)
println("\nCell matrix:")
display(cell_matrix)
println("\nAny NaN values? ", any(isnan.(cell_matrix)))

Periodicity: (true, true, true)


UndefVarError: UndefVarError: `get_cell_vectors` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
system

Atoms(C₂H₅NO₂, periodicity = TTT):
    cell_vectors      : [  33.626        0        0;
                                0   34.122        0;
                                0        0    31.78]u"Å"
    energy            : -57.841278076171875

    AtomView(N,  [ 15.3841,  15.9171,  15.8906]u"Å")
    AtomView(C,  [ 16.8305,   15.912,  15.8893]u"Å")
    AtomView(C,  [ 17.4151,  17.1866,  15.8891]u"Å")
    AtomView(O,  [ 18.6247,  17.3734,  15.8899]u"Å")
    AtomView(O,  [ 16.5734,  18.2167,  15.8914]u"Å")
    AtomView(H,  [ 15.0111,  16.8395,  15.8933]u"Å")
    AtomView(H,  [  14.986,  15.0174,  15.8913]u"Å")
    AtomView(H,  [ 17.1849,  15.3784,  16.7637]u"Å")
    AtomView(H,  [ 17.1803,  15.3538,  14.9953]u"Å")
    AtomView(H,  [ 16.9273,  19.1304,  15.8954]u"Å")
